# Apriori 算法

## 引言：数据挖掘与机器学习
有时候，人们会对机器学习与数据挖掘这两个名词感到困惑。如果你翻开一本冠以机器学习之名的教科书，再同时翻开一本名叫数据挖掘的教材，你会发现二者之间有相当多重合的内容。比如机器学习中也会讲到决策树和支持向量机，而数据挖掘的书里也必然要在决策树和支持向量机上花费相当的篇幅。可见二者确有相当大的重合面，但如果细研究起来，二者也的确是各自不同的领域。

大体上看，数据挖掘可以视为数据库、机器学习和统计学三者的交叉。简单来说，对数据挖掘而言，数据库提供了数据管理技术，而机器学习和统计学则提供了数据分析技术。所以你可以认为数据挖掘包含了机器学习，或者说机器学习是数据挖掘的弹药库中一类相当庞大的弹药集。既然是一类弹药，其实也就是在说数据挖掘中肯定还有其他非机器学习范畴的技术存在。Apriori算法就属于一种非机器学习的数据挖掘技术。

我们都知道数据挖掘是从大量的、不完全的、有噪声的、模糊的、随机的数据中，提取隐含在其中的、人们事先不知道的、但又是潜在有用的信息和知识的过程。 而机器学习是以数据为基础，设法构建或训练出一个模型，进而利用这个模型来实现数据分析的一类技术。这个被训练出来的机器学习模型当然也可以认为是我们从数据中挖掘出来的那些潜在的、有意义的信息和知识。在非机器学习的数据挖掘技术中，我们并不会去建立这样一个模型，而是直接从原数据集入手，设法分析出隐匿在数据背后的某些信息或知识。在后续介绍Apriori算法时，你会相当明显地感受到这一特点。

> **注** apriori 在拉丁语中指 "来自以前"。当定义问题时，通常会使用先验知识或者假设，这被称作 "一个先验" (a priori)。在贝叶斯统计中，使用先验知识作为条件进行推断常见。先验知识可能来自领域知识，先前的一些测量结果，等等。

## Apriori定律
- **Apriori定律1**: 如果一个集合是频繁项集，则它的所有子集都是频繁项集。
  > 例如：假设一个集合{A,B}是频繁项集，即A、B同时出现在一条记录的次数大于等于最小支持度min_support，则它的子集{A},{B}出现次数必定大于等于min_support，即它的子集都是频繁项集。

- **Apriori定律2**: 如果一个集合不是频繁项集，则它的所有超集都不是频繁项集。  
  > 例如：假设集合{A}不是频繁项集，即A出现的次数小于 min_support，则它的任何超集如{A,B}出现的次数必定小于min_support，因此其超集必定也不是频繁项集。

-------------

# Apriori 算法
当集合中项的个数大于0时：

     1. 构建一个由 k 个项组成的候选项集合的列表 
     2. 检查数据以确认每个项集都是频繁的
     3. 保留频繁项集并构建 k+1 项组成的候选项集的列表
    
    

In [14]:
def loadData():
    return [[1,3,4], [2,3,5], [1,2,3,5],[2,5]]

def createC1(dataSet):
    C1=[]
    for transaction in dataSet:
        for item in transaction:
            if not[item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)

def scanD(D,Ck,minSupport):
    ssCnt= {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can):
                    ssCnt[can]=1
                else:
                    ssCnt[can] +=1
    numItems = float(len(D))                    
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
        
    return retList, supportData        

In [8]:
dataSet = loadData()
print dataSet


[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]


In [16]:
C1 = createC1(dataSet)
print C1

[frozenset([1]), frozenset([2]), frozenset([3]), frozenset([4]), frozenset([5])]


In [19]:
D = map(set, dataSet)
print D

[set([1, 3, 4]), set([2, 3, 5]), set([1, 2, 3, 5]), set([2, 5])]


In [23]:
L1, supportData0 = scanD(D, C1, 0.5)
print('L1:', L1)
print('supportData0:', supportData0)

('L1:', [frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])])
('supportData0:', {frozenset([4]): 0.25, frozenset([5]): 0.75, frozenset([2]): 0.75, frozenset([3]): 0.75, frozenset([1]): 0.5})


In [24]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
                
    return  retList              
    

In [27]:
def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)
    D = map(set,dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, SupK = scanD(D, Ck, minSupport)
        supportData.update(SupK)
        L.append(Lk)
        k +=1
    return L, supportData     

In [29]:
L, supportData = apriori(dataSet)
L

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})],
 [frozenset({2, 3, 5})],
 []]

In [31]:
L[1]

[frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})]

In [32]:
L[2]

[frozenset({2, 3, 5})]

In [33]:
L[3]

[]

In [34]:
aprioriGen(L[0], 2)

[frozenset({1, 3}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({3, 5}),
 frozenset({2, 5})]

In [35]:
L, supportData = apriori(dataSet, minSupport=0.7)
L

[[frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({2, 5})], []]

## 从频繁项中挖掘关联规则



In [39]:
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if(i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
                
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
                
    return bigRuleList

def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    pruneH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if (conf >= minConf):
            print(freqSet-conseq, '--->', conseq, 'conf:', conf)
            br1.append((freqSet-conseq, conseq, conf))
            pruneH.append(conseq)
    return pruneH


def rulesFromConseq(freqSet, H, supportData, br1, minConf=.7):
    m = len(H[0])
    if(len(freqSet) > (m+1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if(len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

In [43]:
L, supportData = apriori(dataSet, minSupport=0.5)
rules = generateRules(L, supportData, minConf=0.7)
rules

(frozenset([1]), '--->', frozenset([3]), 'conf:', 1.0)
(frozenset([5]), '--->', frozenset([2]), 'conf:', 1.0)
(frozenset([2]), '--->', frozenset([5]), 'conf:', 1.0)


[(frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0)]

In [44]:
rules = generateRules(L, supportData, minConf=0.5)
rules

(frozenset([3]), '--->', frozenset([1]), 'conf:', 0.6666666666666666)
(frozenset([1]), '--->', frozenset([3]), 'conf:', 1.0)
(frozenset([5]), '--->', frozenset([2]), 'conf:', 1.0)
(frozenset([2]), '--->', frozenset([5]), 'conf:', 1.0)
(frozenset([3]), '--->', frozenset([2]), 'conf:', 0.6666666666666666)
(frozenset([2]), '--->', frozenset([3]), 'conf:', 0.6666666666666666)
(frozenset([5]), '--->', frozenset([3]), 'conf:', 0.6666666666666666)
(frozenset([3]), '--->', frozenset([5]), 'conf:', 0.6666666666666666)
(frozenset([5]), '--->', frozenset([2, 3]), 'conf:', 0.6666666666666666)
(frozenset([3]), '--->', frozenset([2, 5]), 'conf:', 0.6666666666666666)
(frozenset([2]), '--->', frozenset([3, 5]), 'conf:', 0.6666666666666666)


[(frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

一旦降低可信度阀值，就可以获得更多的规则。